In [1]:
import os
from thermof.read import read_log, read_thermo
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read Scan thermo

In [ ]:
SCAN = 'DC_Cu110'
SCANDIR = '/home/kutay/Documents/git/Nanocar/surface-scan'
DATADIR = os.path.join(SCANDIR, 'analysis', 'data', SCAN)
scanlist = [os.path.join(DATADIR, i) for i in os.listdir(DATADIR)]

In [ ]:
thermo_headers = 'Step Temp Press TotEng E_pair E_mol Fmax Fnorm c_C1[1] c_C1[2] c_C1[3]'
thermo_keys = ['step', 'temp', 'press', 'etotal', 'epair', 'emol', 'fmax', 'fnorm', 'x', 'y', 'z']

### Read LAMMPS log files

In [ ]:
xidx = 10
data = {'x': np.zeros(ny), 'y': np.zeros(ny), 'z': np.zeros(ny), 'x0': np.zeros(ny), 'y0': np.zeros(ny), 'ypos': np.zeros((ny, 1001)),
        'emol': np.zeros(ny), 'epair': np.zeros(ny), 'etotal': np.zeros(ny), 'fmax': np.zeros(ny), 'fnorm': np.zeros(ny)}
for simdir in scanlist:
    xi, yi = [int(i) for i in os.path.basename(simdir).split('-')]
    if xi == xidx:
        logfile = os.path.join(simdir, 'log.%s' % SCAN)
        thermo_data = read_log(logfile, headers=thermo_headers)
        thermo = read_thermo(thermo_data, headers=thermo_keys)[0]
        for k in ['x', 'y', 'z', 'epair', 'emol']:
            data[k][yi] = np.mean(thermo[k])
        data['x0'][yi] = thermo['x'][0]
        data['y0'][yi] = thermo['y'][0]
        data['ypos'][yi] = thermo['y']

In [ ]:
def subplot(plot_fun, plot_args, nrow=1, width=4, height=3, dpi=200, save=None):
    n_plots = len(plot_args)
    ncol = np.ceil(n_plots / nrow)
    figsize = (ncol * width, nrow * height)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(hspace=.5, wspace=.25)
    for idx, args in enumerate(plot_args, start=1):
        args['ax'] = fig.add_subplot(nrow, ncol, idx)
        plot_fun(**args)
    if save is not None:
        plt.savefig(save, dpi=dpi, transparent=True, bbox_inches='tight')

def scatter_plot(x, y, ax, title='', xlabel='', ylabel=''):
    ax.scatter(x, y,  c='xkcd:crimson', s=100, alpha=0.6)
    ax.set_title(title)
    ylims = list(ax.get_ylim())
    ax.plot([22.8912, 22.8912], ylims, c='k')
    ax.plot([25.4346, 25.4346], ylims, c='k')
    ax.plot([24.1629, 24.1629], ylims, 'k--', alpha=0.7)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

In [ ]:
start =  [23.3805, 24.1629, 10]          # Molecule starting position
dx, dy, buffer = 0.2, 0.1, 0.2
xdist, ydist = 2.0, 1.5
ypos = np.arange(start[1] - ydist - buffer, start[1] + ydist + buffer + dy, dy)
plot_data = []
for yvar in ['x0', 'x', 'y0', 'y', 'z', 'epair']:
    plot_data.append(dict(x=ypos, y=data[yvar], title=yvar, xlabel='y (Å)'))

In [ ]:
surfatoms1 = [[21.582, 22.8912], [25.179, 22.8912], [21.582, 25.4346], [25.179, 25.4346]]
surfatoms2 = [23.3805, 24.1629]

In [ ]:
subplot(scatter_plot, plot_data)

### Position histogram

In [ ]:
from scipy.stats import norm
import matplotlib.mlab as mlab

In [ ]:
def plot_hist(data, bins=50, color='b', linestyle='k', alpha=0.7, lw=1.5):
    (mu, sigma) = norm.fit(data)
    # the histogram of the data
    n, bins, patches = plt.hist(data, bins, normed=1, facecolor=color, alpha=alpha)
    # add a 'best fit' line
    y = mlab.normpdf(bins, mu, sigma)
    l = plt.plot(bins, y, linestyle, linewidth=lw)
    plt.scatter(data[0], max(y), c='k')

In [ ]:
histdata = data['ypos']
fig = plt.figure(figsize=(16, 2.5), dpi=300)
for hd in histdata:
    plot_hist(hd, alpha=0.25, color='xkcd:crimson')
plt.xlabel('y (Å)')
# pltfile = os.path.join('/home/kutay/Documents/git/Nanocar/docs/assets/img/dctst', '%s-hist.png' % SCAN)
# plt.savefig(pltfile, dpi=300, transparent=True, bbox_inches='tight')
# plt.yticks([0, 0.5, 1.0, 1.5])
# plt.xlim(22, 26.5)
# plt.ylim(0, 1.5)